In [1]:
from nltk.corpus import brown
from nltk.corpus import stopwords
import string
from collections import Counter
import operator
import numpy as np
from math import log10
from sklearn import decomposition

In [2]:
clean_words = [word.lower() for word in brown.words()]

In [3]:
clean_words = [''.join(c for c in s if c not in string.punctuation) for s in clean_words]

In [4]:
clean_words = [s for s in clean_words if s]

In [5]:
clean_words = [s for s in clean_words if len(s) > 1]


In [6]:
filtered_words = [word for word in clean_words if word not in stopwords.words('english')]

In [8]:
clean_count = Counter(filtered_words)
clean_sort = sorted(clean_count, key=clean_count.get, reverse=True)


In [9]:
top5000=clean_sort[:5000]
top1000=clean_sort[:1000]

In [188]:

for index, word in enumerate(filtered_words):
    for value in top5000:
        if value == word:
            for phrase in top1000:
                n[value, phrase] = filtered_words[index-window: index+window].count(phrase)
                sum_n += n[value, phrase]
        else:
            pass
    

In [29]:
window = 2
n ={}
sum_n = 0
for value in top5000:
    indexes = [i for i,x in enumerate(filtered_words) if x ==  value]
    for phrase in top1000:
        n[value, phrase] = 0
        for index in indexes:
            index_count = filtered_words[index-window: index+window+1].count(phrase)
            n[value, phrase] += index_count
        sum_n += n[value, phrase]

In [31]:
temp={k:v for (k,v) in n.items() if v >1 }
len(temp)

115187

###Construct P(W), P(C|W) and P(C)

P(C|W) = p(c&W)/p(W)

In [34]:
clean_counted = dict(clean_count)

In [35]:
sum5000 = 0
for word in top5000:
     sum5000 += clean_counted[word]

In [36]:
sum1000 = 0
for word in top1000:
     sum1000 += clean_counted[word]

In [37]:
p_w = {}
p_c = {}
p_cw = {}
p_n = {}

In [38]:
for word in top5000:
    p_w[word] = clean_counted[word]/sum5000
    for phrase in top1000: 
        p_n[word, phrase] = n[word, phrase]/sum_n
        p_cw[word, phrase] = p_n[word, phrase]/p_w[word]

In [39]:
for word in top1000:
    p_c[word] = clean_counted[word]/sum1000

###Represent each vocabulary item w by a |C|-dimensional vector Φ(w)

In [40]:
phi ={}
for word in top5000:
    for phrase in top1000: 
        if p_cw[word,phrase] > 0:
            phi[word, phrase] = max(0, log10(p_cw[word,phrase]/p_c[phrase]))
        else:
            phi[word, phrase] = 0

### Generate Matrix

In [41]:
matrix = [[0 for x in range(len(top1000))] for y in range(len(top5000))] 
for i in range(len(top5000)):
    for j in range(len(top1000)):
        word = top5000[i]
        phrase =  top1000[j]
        matrix[i][j] = phi[(word,phrase)]


In [42]:
matrix2 = np.asarray(matrix)

In [43]:
pca = decomposition.PCA(n_components=100)

In [44]:
pca.fit(matrix2)

PCA(copy=True, n_components=100, whiten=False)

In [45]:
X = pca.transform(matrix2)

In [46]:
X.shape

(5000, 100)

In [47]:
print(pca.explained_variance_ratio_) 

[ 0.01499068  0.0095628   0.00738975  0.00632543  0.005073    0.0046134
  0.00358666  0.00349249  0.00299991  0.00298792  0.00288307  0.0027975
  0.002689    0.00265088  0.00260458  0.00253232  0.00251749  0.00243131
  0.00236754  0.00234594  0.00233501  0.00230606  0.00226343  0.00224178
  0.00220825  0.00218363  0.00217137  0.00215188  0.00213567  0.00213335
  0.00210333  0.00209003  0.00208253  0.00207453  0.00206783  0.00205622
  0.00205256  0.002046    0.00202746  0.00202311  0.00201184  0.00200502
  0.00200244  0.00198783  0.00197959  0.00197335  0.00196593  0.0019573
  0.00194911  0.00194604  0.00193238  0.0019279   0.00192595  0.00191952
  0.00191696  0.00190576  0.00190449  0.00189918  0.00189692  0.00188564
  0.00187846  0.00187339  0.00186642  0.00186006  0.00185432  0.001848
  0.00184494  0.00184157  0.00183497  0.00182997  0.00182679  0.00182063
  0.00181504  0.00180358  0.00179965  0.00179548  0.00178812  0.00178083
  0.00177897  0.00177625  0.00177337  0.00176617  0.0017

In [48]:
print(pca.explained_variance_ratio_.cumsum()) 

[ 0.01499068  0.02455349  0.03194323  0.03826867  0.04334167  0.04795506
  0.05154172  0.05503421  0.05803412  0.06102203  0.06390511  0.06670261
  0.06939161  0.07204248  0.07464706  0.07717938  0.07969687  0.08212817
  0.08449572  0.08684165  0.08917667  0.09148273  0.09374615  0.09598793
  0.09819618  0.10037981  0.10255118  0.10470305  0.10683873  0.10897207
  0.11107541  0.11316544  0.11524796  0.11732249  0.11939033  0.12144655
  0.12349911  0.12554511  0.12757257  0.12959568  0.13160752  0.13361255
  0.13561498  0.13760282  0.13958241  0.14155576  0.14352169  0.14547899
  0.14742809  0.14937414  0.15130652  0.15323443  0.15516038  0.1570799
  0.15899685  0.16090261  0.1628071   0.16470628  0.16660319  0.16848883
  0.17036729  0.17224068  0.17410711  0.17596717  0.17782149  0.17966948
  0.18151442  0.18335599  0.18519096  0.18702092  0.18884772  0.19066835
  0.19248339  0.19428697  0.19608662  0.19788209  0.19967021  0.20145104
  0.20323001  0.20500626  0.20677964  0.2085458   0.

###Clustering

In [49]:
from sklearn.cluster import KMeans

In [50]:
from scipy.cluster.vq import kmeans2

In [51]:
#random_State = 0 k means pluis plus 
kmeans = KMeans( n_clusters = 100 ).fit(matrix2)

In [52]:
#random_State = 0 k means pluis plus
centroids, idx = kmeans2(matrix2, k = 100,  minit='random')

/usr/local/lib/python3.4/site-packages/scipy/cluster/vq.py:648: UserWarning: One of the clusters is empty. Re-run kmean with a different initialization.
  warnings.warn("One of the clusters is empty. "


In [53]:
top5000words = np.array(top5000)

In [54]:
for cluster_id in np.unique(idx):
    #exemplar = top5000[kmeans.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(top5000words[np.asarray(np.nonzero(idx==cluster_id)).flatten()])
    cluster_str = ", ".join(cluster)
    print(cluster_str)
    #print(" - *%s:* %s" % (exemplar, cluster_str))

broad, cheek, fantastic, illness, imagination, pencil, sides
adjustments, advertising, area, available, basis, benefit, commercial, construction, cooling, cost, designed, desired, determining, developments, disposal, economy, efficient, electronic, employment, equipment, evaluation, existing, facilities, farm, gin, hunting, improved, improvements, longrange, machine, machinery, marketing, materials, methods, observed, operation, overall, parttime, plant, potential, processes, processing, production, products, protect, provide, providing, purposes, radio, recreation, reduce, rural, sba, shelter, site, standard, stations, structural, suitable, supplies, techniques, testing, urban, use, utility, vehicles
absorbed, arts, attacked, automatically, begins, cholesterol, christians, conductor, contacts, covers, hypothalamus, increasingly, listened, memorial, municipal, obliged, post, prepare, recommended, salary, suspected, universal, wealth
absolute, acting, allow, apparatus, applied, applies,

###Nearest Neighbor Results

In [75]:
subW = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind', 'africa', 'chicago', 'revolution', 'september',
'chemical', 'detergent', 'dictionary', 'storm', 'worship','harvard','gun','war','mother','japanese','chinese','wisconsin'
       ,'killer','cancer','bible','republican','democrats']

In [76]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from sklearn.metrics import euclidean_distances

In [77]:
d = {}
closet = {}
for word in subW:
    d[word] = 10
    closet[word] = 'ccc'
    for w in top5000: 
        if w != word:
            temp_d = cosine(X[top5000.index(word)], X[top5000.index(w)])
            #print(temp_d)
            if abs(temp_d) < d[word]:
                d[word] = temp_d
                closet[word] = w

In [78]:
closet

{'africa': 'asia',
 'autumn': 'late',
 'bible': 'experienced',
 'cancer': 'harvard',
 'chemical': 'drugs',
 'chicago': 'portland',
 'chinese': 'peoples',
 'cigarette': 'lighted',
 'communism': 'utopian',
 'democrats': 'liberal',
 'detergent': 'fabrics',
 'dictionary': 'text',
 'gun': 'back',
 'harvard': 'professor',
 'japanese': 'stress',
 'killer': 'guys',
 'mankind': 'nation',
 'mother': 'son',
 'pulmonary': 'artery',
 'republican': 'party',
 'revolution': 'developed',
 'september': 'june',
 'storm': 'noon',
 'war': 'world',
 'wisconsin': 'carolina',
 'worship': 'religious'}